# <span style="color:green"> Numerical Simulation Laboratory (NSL) </span>
## <span style="color:blue">  Numerical exercises 4</span>

### Exercise 04.1

The Molecular Dynamics / Monte Carlo (MD_MC) code we have provided compute the total (internal) energy per particle, $E/N$, the potential energy per particle, $U/N$, the kinetic energy per particle, $K/N$, and the temperature, $T$.

<span style="color:red">Include the calculation of the pressure, $P$, inside your Molecular Dynamics code: block averages, progressive average values and progresssive uncertainties.</span>

Remember that, via the **virial theorem**, the pressure in a Lennard-Jones system can be estimated by means of:

$$ P = \rho k_B T + \frac{1}{3V} \left\langle \sum_{i=1}^{N-1} \sum_{j\,(>i)}^N 48\epsilon \left[ \left(\frac{\sigma}{|\vec{r}_i -\vec{r}_j|}\right)^{12} - 
\frac{1}{2} \left(\frac{\sigma}{|\vec{r}_i -\vec{r}_j|}\right)^6 \right] \right\rangle $$

---

Si studia un sistema di $108$ particelle disposte all'interno di un box con condizioni periodiche al contorno e interagenti entro una distanza di cut-off mediante un potenziale di tipo Lennard-Jones della forma:

$$V(r)=4\epsilon\Bigg[\bigg(\frac{\sigma}{r}\bigg)^{12}-\bigg(\frac{\sigma}{r}\bigg)^6\Bigg].$$

Noto come l'energia potenziale, in questo caso, oltre ad avere una forma complicata, è una funzione delle posizioni delle molecole del sistema. Per questo motivo non esiste una soluzione analitica alle equazioni del moto, e dunque l'unico modo per arrivare ad un risultato è integrarle in modo numerico.

Questo caso di dinamica molecolare viene simulato attraverso l'algoritmo di Verlet, che si basa sull'integrazione delle equazioni del moto per determinare la posizione e la velocità di un certo numero di particelle in un dato istante temporale.

In particolare tale algoritmo computa le posizioni delle particelle come

$$\vec{r}(t+\delta t)=2\vec{r}(t)-\vec{r}(t-\delta t) + \vec{a}(t)\delta t^2\,\,,$$

per cui tale algoritmo non necessita solo delle posizioni attuali, ma anche di quelle dello step precedente.

L'algoritmo di Verlet dunque non utilizza le velocità per computare le posizioni delle particelle passo dopo passo, che vengono calcolate attraverso

$$\vec{v}(t) = \frac{\vec{r}(t+\delta t) - \vec{r}(t-\delta t)}{2\delta t}\,\,,$$

ma queste risultano utili per calcolare l'energia cinetica del sistema, e quindi anche la temperatura.

Le quantità che si vuole calcolare per quanto riguarda la dinamica molecolare in questione, oltre la pressione $P$, sono le seguenti:
- Energia potenziale: $U = 4\sum_{i=1}^{N-1}\sum_{j > i}^N \Big[\Big(\frac{1}{|\vec{r}_i-\vec{r}_j|}\Big)^{12} - \Big(\frac{1}{|\vec{r}_i-\vec{r}_j|}\Big)^{6}\Big]$
- Energia cinetica: $K = \frac{1}{2}\sum_{i=1}^{N} |\vec{v}_i|^2$
- Energia totale: $E= 4U+K$
- Temperatura: $T = \frac{2}{3N}K$

E in particolare vengono calcolate le medie e le deviazioni standard progressive di dei singoli contributi delle $N=108$ sulle osservabili particelle all'interno degli $M$ blocchi. Il procedimento per la stima di tali osservabili è il seguente:

1. La configurazione di partenza del singolo blocco è determinata dalle molecole del sistema in uno stato cristallino, e il programma legge da un file esterno le caratteristiche del sistema quali ad esempio il numero di particelle $N$ per blocco, la densità $\rho$ del sistema (da cui si può calcolare il volume del blocco), il passo dell'algoritmo $\delta t$ e la temperatura. Quest'ultima serve per estrarre dalla distribuzione gaussiana $\mathcal{G}(0, T^{1/2})$ le velocità iniziali $\vec{v}(t=0)$ delle particelle. In questo modo si possono calcolare le posizioni di queste al tempo $-\delta t$, in modo da rendere l'algoritmo di Verlet simmetrico temporalmente, attarverso

$$\vec{r}(-\delta t) = \vec{r}(0) - \vec{v}(0)\delta t\,\,.$$

2. Dunque dopo aver misurato le osservabili di partenza, si implementa un passo dell'algoritmo di Verlet, ovvero si computa la forza che il sistema imprime sulla singola particella, determinata dall'opposto del gradiente del potenziale di Lennard-Jones $F=-\nabla U\,\,.$ In questo modo possono essere calcolate le nuove coordinate e le nuove velocità delle particelle come
$$\vec{r}(t+\delta t)=2\vec{r}(t)-\vec{r}(t-\delta t) + \vec{a}(t)\delta t^2\,\,\,\,,\,\,\,\,\vec{v}(t) = \frac{\vec{r}(t+\delta t) - \vec{r}(t-\delta t)}{2\delta t}\,\,.$$


3. Dopodichè, dopo ogni passo si misurano le osservabili e si accumulano in un vettore, da cui poi se ne ricaverà la media dopo un certo numero di passi.


4. Infine, iterando lo stesso procedimento su $M$ blocchi, è possibile calcolare la media progressiva e la deviazione standard progressiva di tutte le osservabili.

In [1]:
import matplotlib
import matplotlib.pyplot as plt
import numpy as np
import math

risultati_T = np.loadtxt('output_temp.dat')
blocchi = risultati_T[:,0]
medie_T = risultati_T[:,2]
deviazioni_T = risultati_T[:,3]

risultati_U = np.loadtxt('output_epot.dat')
medie_U = risultati_U[:,2]
deviazioni_U = risultati_U[:,3]

risultati_K = np.loadtxt('output_ekin.dat')
medie_K = risultati_K[:,2]
deviazioni_K = risultati_K[:,3]

risultati_E = np.loadtxt('output_etot.dat')
medie_E = risultati_E[:,2]
deviazioni_E = risultati_E[:,3]

risultati_P = np.loadtxt('output_pres.dat')
medie_P = risultati_P[:,2]
deviazioni_P = risultati_P[:,3]


plt.errorbar(blocchi, medie_T, yerr=deviazioni_T)
plt.axhline(y=1.1, color='red', linestyle='--', label='Temperatura attesa')
plt.xlabel('Blocchi')
plt.ylabel('Temperatura')
# plt.title('Temperatura')
plt.grid(True)
plt.legend()
plt.show()

plt.errorbar(blocchi, medie_U, yerr=deviazioni_U)
plt.xlabel('Blocchi')
plt.ylabel('Energia potenziale per particella')
plt.grid(True)
plt.show()

plt.errorbar(blocchi, medie_K, yerr=deviazioni_K)
plt.xlabel('Blocchi')
plt.ylabel('Energia cinetica per particella')
plt.grid(True)
plt.show()

plt.errorbar(blocchi, medie_E, yerr=deviazioni_E)
plt.xlabel('Blocchi')
plt.ylabel('Energia totale per particella')
plt.grid(True)
plt.show()

plt.errorbar(blocchi, medie_P, yerr=deviazioni_P)
plt.xlabel('Blocchi')
plt.ylabel('Pressione')
plt.grid(True)
plt.show()

FileNotFoundError: output_temp.dat not found.

### Exercise 04.2

By means of your upgraded code, <span style="color:red">perform MD simulations via a Lennard-Jones model</span> of Argon ($\sigma = 0.34$ nm, $\epsilon/k_B = 120$ K, $m=39.948$ amu) in the following conditions:
1. solid phase: $\rho^\star = 1.1$, $T^\star = 0.8$ (cut-off radius: $r_c = 2.2$)
2. liquid phase: $\rho^\star = 0.8$, $T^\star = 1.1$ (cut-off radius: $r_c = 2.5$)
3. gas phase: $\rho^\star = 0.05$, $T^\star = 1.2$ (cut-off radius: $r_c = 5.0$)

and show in pictures the obtained progressive average values and uncertainties as a functions of the increasing number of blocks for the total (internal) energy per particle, $E/N$, the potential energy per particle, $U/N$, the kinetic energy per particle, $K/N$, the temperature, $T$, and the pressure, $P$, in reduced units!

### <span style="color:green"> Equilibration is needed!</span>

By monitoring the temperature, $T$, during the first time steps of the simulation, you will observe that at the beginning, starting from a perfect crystalline configuration, $T$ will not conserve its value. Can you understand and explain this behaviour?

Explore and find the starting value of $T$ needed to stabilize, after a certain number of time steps (equilibration time), the average value of $T$ on the desired $T^\star$. <span style="color:red">Perform your solid/liquid phase MD simulations by re-starting them after such equilibration time.</span>